# Déflexion d'une plaque mince encastrée avec FEniCSx

Ce notebook simule la déflexion d'une plaque carrée encastrée soumise à une charge uniforme, selon la théorie de Kirchhoff-Love.

In [ ]:
import numpy as np
import ufl
from mpi4py import MPI
from dolfinx import mesh, fem
from basix.ufl import element
import pyvista
pyvista.start_xvfb()

In [ ]:
# Paramètres physiques
L = 1.0
n = 32
E = 210e9
nu = 0.3
t = 0.01
q0 = 1000
D = E * t**3 / (12 * (1 - nu**2))

In [ ]:
# Maillage triangulaire
domain = mesh.create_unit_square(MPI.COMM_WORLD, n, n)
V = fem.FunctionSpace(domain, element("Lagrange", "triangle", 2))

In [ ]:
# Formulation variationnelle
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = D * ufl.inner(ufl.grad(ufl.grad(u)), ufl.grad(ufl.grad(v))) * ufl.dx
L_form = q0 * v * ufl.dx

In [ ]:
# Conditions aux limites encastrées
def clamped_boundary(x):
    return np.isclose(x[0], 0) | np.isclose(x[0], 1) | np.isclose(x[1], 0) | np.isclose(x[1], 1)

fdim = domain.topology.dim - 1
facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)
dofs = fem.locate_dofs_topological(V, fdim, facets)
bc = fem.dirichletbc(fem.Constant(domain, 0.0), dofs, V)

In [ ]:
# Résolution
problem = fem.petsc.LinearProblem(a, L_form, bcs=[bc])
uh = problem.solve()

In [ ]:
# Visualisation
from dolfinx.plot import create_vtk_mesh
grid = pyvista.UnstructuredGrid(*create_vtk_mesh(V))
grid.point_data["w"] = uh.x.array.real
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True, scalars="w", cmap="viridis")
plotter.view_xy()
plotter.show()